## Welcome to a guide on the Enigma Machine!

### The Enigma Machine was an encryption device used by the German Military during WWII.

An Enigma Machine is made up of (usually 3) rotors, a reflector and (an optional) reflector.
Each part performs a substituion cipher (swaps one letter with another) but all do it in different ways.  
Let's take a look at an Enigma Machine.

In [14]:
from enigma_machine import EnigmaMachine
# Create an Enigma Machine with default parameters.
print(EnigmaMachine())

Enigma Machine with 3 rotors, a reflector and a plugboard.


Let's encrypt our first message.

In [15]:
EM = EnigmaMachine()
encrypted_message = EM.encrypt_message('HELLO WORLD!')
print(encrypted_message)

SDHOJ GSMAY!


You can decrypt a message by just entering it again in the same machine. We'll have to initialise a new machine though as the settings change as soon as you type something! (More on that later).

In [3]:
EM = EnigmaMachine()
decrypted_message = EM.encrypt_message('SDHOJ GSMAY!')
print(decrypted_message)

HELLO WORLD!


The convention for encrypted messages was blocks of 5 letters with no spaces, and "X" in places of full stops.
Here's an example.

In [16]:
EM = EnigmaMachine()
EM.encrypt_message('UCDKN DITSL ERDPS YIIF')

'THISI SATES TMESS AGEX'

Removing the spaces: THISISATESTMESSAGEX  
Interpreting the message: THIS IS A TEST MESSAGE.

Let's have a go at decrypting a message actually sent using a real world Enigma machine.
This is from the Enigma training manual from 1930.  
As well as an X replacing a full stop, "Q" replaces "CH" and "AE" for "Ä", "UE" for "Ü" and "Z" for "ß"). 

We will need to configure the machine with the correct settings- these will be explained in a bit more detail later.  
Machine Settings:  
Reflector: A  
Wheel order: II I III  
Ring positions: 	24 13 22  
Plug pairs:	AM FI NV PS TU WZ  

GCDSE AHUGW TQGRK VLFGX UCALX VYMIG MMNMF DXTGN VHVRM MEVOU YFZSL RHDRR XFJWC FHUHM UNZEF RDISI KBGPM YVXUZ

In [17]:
# Configuring the machine.
EM = EnigmaMachine(rotor_types=['II', 'I', 'III'],
                    reflector_mapping='A',
                    rotor_positions='ABL',
                    ring_settings='XMV',
                    steckered_pairing='AM FI NV PS TU WZ')

EM.encrypt_message('GCDSE AHUGW TQGRK VLFGX UCALX VYMIG MMNMF DXTGN VHVRM MEVOU YFZSL RHDRR XFJWC FHUHM UNZEF RDISI KBGPM YVXUZ')

'FEIND LIQEI NFANT ERIEK OLONN EBEOB AQTET XANFA NGSUE DAUSG ANGBA ERWAL DEXEN DEDRE IKMOS TWAER TSNEU STADT'

Removing spaces and replacing special characters:
FEINDLICHEINFANTERIEKOLONNEBEOBACHTET.ANFANGSUEDAUSGANGBAERWALDE.ENDEDREIKMOSTWAERTSNEUSTADT

Splitting out into German words:
Feindliche Infanterie kolonne beobachtet. Anfang Südausgang Bärwalde. Ende 3km ostwärts Neustadt.

Translation:
Enemy infantry column was observed. Beginning southern exit Baerwalde. Ending 3km east of Neustadt.

## How does it work?

The encryption made by an Enigma machine is made up of three components:  
- Rotors  
- Reflector  
- Plugboard  

When a key is pressed, a current passes through these parts in turn:  
1) Plugboard  
2) Rotors (flowing right to left)  
3) Reflector  
4) Rotors (in reverse, left to right)  
5) Plugboard  
The output is then displayed by one of the letters being lit up on the 'lampboard'.  
Let's take a further look at these constituent parts.  


## Rotor
The rotor is the most important part of an Enigma Machine. It is a wheel of 26 letters and performs a basic letter substitution.  
This substitution works by telling us which letter of the alphabet is mapped to which.  
There are only a few different set mappings to choose from (called the rotor "type") each denoted by a Roman Numeral (I, II, and so on).  
In the case below A->E, B->K, C->M and so on.  


In [6]:
# Build a rotor with default values
rotor = EM.Rotor()

print(rotor.mapping)

EKMFLGDQVZNTOWYHXUSPAIBRCJ


Within the Enigma Machine each rotor can have one of 26 positions (known as *Grundstellung*), one for each letter of the alphabet. You can change this position by turning (essentially rotating) the rotor.

In [7]:
rotor = EM.Rotor()
print(f'Rotor position before turning: {rotor.position}')
rotor.turn_rotor()
print(f'Rotor position after turning: {rotor.position}')

Rotor position before turning: A
Rotor position after turning: B


The rotor's position changes how it maps letters. For example take rotor I which has the mapping EKMFLGDQVZNTOWYHXUSPAIBRCJ.

If a rotor is in "A" position, the mapping is the same as before A->E, B->K, C->M, D->F.  
If a rotor is in "B" position, it has been rotated along by one, so the mapping is shifted one place, so A->K, B->M, C->F.


### How does a rotor get turned?  
Enigma Machines contain a set of (usually 3) rotors which interconnect, and every time a key is pressed, the right-most rotor is always rotated.  
Each rotor also has a notch at a position on its wheel. If the rotor is its notch position, it turns the rotor immediately to its left.


In [8]:
EM = EnigmaMachine(rotor_positions='AAU')
for rotor in EM.rotors:
    print(f'Position: {rotor.position}, notch {rotor.notch}')
print('Turning rotor assembly...')
EM.turn_rotor_assembly()
for rotor in EM.rotors:
    print(f'Position: {rotor.position}, notch {rotor.notch}')
print('Turning rotor assembly...')
EM.turn_rotor_assembly()
for rotor in EM.rotors:
    print(f'Position: {rotor.position}, notch {rotor.notch}')

Position: A, notch Q
Position: A, notch E
Position: U, notch V
Turning rotor assembly...
Position: A, notch Q
Position: A, notch E
Position: V, notch V
Turning rotor assembly...
Position: A, notch Q
Position: B, notch E
Position: W, notch V


In the first turn, the right-most (last) rotor is turned, so U->V.
This rotor is now at notch position, so in the next turn it rotates the second rotor from A->B.

It takes 26x25x26 = 16,900 key presses to get back to the initial rotor positions. (Note not 26x26x26 due to a phenomenon called "double-stepping").

### Ring settings
The final property of an Enigma rotor is its ring settings (called *Ringstellung*). Like the rotor position, there are 26 ring different settings and it changes the way the mapping of the rotor works.  
When the ring setting is set, it essentially turns round the internal wiring of the rotor which has the same effect as turning the rotor, but also shifts all the letters in the mapping up the alphabet a certain amount.  
If the ring setting is at 'B', this shifts all the letters along the mapping by 1, but also shifts them up the alphabet by 1.  
For example: 

In [20]:
rotor = EM.Rotor(rotor_type='II', ring_setting='A')
print(f'Rotor mapping ring setting "A": {rotor.mapping}')
rotor.apply_ring_setting('B')
print(f'Rotor mapping ring setting "B": {rotor.mapping}')

Rotor mapping ring setting "A": AJDKSIRUXBLHWTMCQGZNPYFVOE
Rotor mapping ring setting "B": FBKELTJSVYCMIXUNDRHAOQZGWP


Each letter in the mapping has been bumped along one and shifted up the alphabet by 1.

Putting it all together, a rotor has many different configurable settings:  
- Type (e.g. I, II, III) the underlying substitution cipher of the rotor
- Notch (single letter) controls when the rotor to the left is turned
- Ring setting (single letter) defines how the internal wiring is set up and changes the substitution
- Rotor position (single letter) specifies the rotational position of the rotor in the machine (changes when keys are typed)

Only the rotor position changes by using the machine, all others are set at the beginning and remain unchanged.

## Reflector
The reflector (known as *Umkehrwalze*) is a device that maps all the alphabet into pairs of letters. It enables the same machine to both encrypt and decrypt a message without changing the settings. However, it led to a key cryptographic weakness that the codebreakers at Bletchley Park were able to exploit.  
There were 3 standard reflectors for the Enigma Mk 1, 'A', 'B' and 'C'.

In [10]:
reflector = EnigmaMachine.Reflector('A')
print(reflector)
reflector = EnigmaMachine.Reflector('B')
print(reflector)
reflector = EnigmaMachine.Reflector('C')
print(reflector)

A reflector for an Enigma Machine with mapping "EJMZALYXVBWFCRQUONTSPIKHGD".
A reflector for an Enigma Machine with mapping "YRUHQSLDPXNGOKMIEBFZCWVJAT".
A reflector for an Enigma Machine with mapping "FVPJIAOYEDRZXWGCTKUQSBNMHL".


## Plugboard
The plugboard (known as *Steckerbrett*) is a way of pairing up (usually 10) letters. It looked a lot like old style phone switchboards, with a socket for each letter, and a cable plugged into two sockets making a pairing.  
This mapping is done at the start and end of the process. Without the plugboard, the Enigma cipher was solvable by hand, but its introduction led codebreakers to develop the first computers in order to break the code.  
The letter pairings of the plugboard are called "steckered pairings".

In [11]:
plugboard = EnigmaMachine.Plugboard()
print(plugboard)

A plugboard for an Enigma Machine with steckered pairing "AM FI NV PS TU WZ".


Let's look at another example of text that was actually sent using an Enigma Machine now we know what the settings mean.

This was a message sent from the Wermacht during Operation Barbarossa (invasion of the Soviet Union) from 1941.  

'EDPUD NRGYS ZRCXN UYTPO MRMBO FKTBZ REZKM LXLVE FGUEY SIOZV EQMIK UBPMM YLKLT TDEIS MDICA GYKUA CTCDO MOHWX MUUIA UBSTS LRNBZ SZWNR FXWFY SSXJZ VIJHI DISHP RKLKA YUPAD TXQSP INQMA TLPIF SVKDA SCTAC DPBOP VHJK-'  

The set up of the Enigma Machine was as follows:  
Reflector: B  
Wheel order: II IV V  
Ring positions: BUL  
Plug pairs:	AV BS CG DL FU HZ IN KM OW RX
Rotor positions: BLA
Let's decrypt it!


In [21]:
# Configure Enigma Machine
EM = EnigmaMachine(rotor_types=['II', 'IV', 'V'],
                   rotor_positions='BLA',
                   ring_settings='BUL',
                   steckered_pairing='AV BS CG DL FU HZ IN KM OW RX',
                   reflector_mapping='B')

message = 'EDPUD NRGYS ZRCXN UYTPO MRMBO FKTBZ REZKM LXLVE FGUEY SIOZV EQMIK UBPMM YLKLT TDEIS MDICA GYKUA CTCDO MOHWX MUUIA UBSTS LRNBZ SZWNR FXWFY SSXJZ VIJHI DISHP RKLKA YUPAD TXQSP INQMA TLPIF SVKDA SCTAC DPBOP VHJK-'

print(EM.encrypt_message(message))

AUFKL XABTE ILUNG XVONX KURTI NOWAX KURTI NOWAX NORDW ESTLX SEBEZ XSEBE ZXUAF FLIEG ERSTR ASZER IQTUN GXDUB ROWKI XDUBR OWKIX OPOTS CHKAX OPOTS CHKAX UMXEI NSAQT DREIN ULLXU HRANG ETRET ENXAN GRIFF XINFX RGTX-


It doesn't look that clear yet, but if we replace special characters and sort the spaces we get:  
AUFKL ABTEILUNG VON KURTINOWA KURTINOWA NORDWESTL SEBEZ SEBEZ UAFFLIEGERSTR ASßERICHTUNG DUBROWKI DUBROWKI OPOTSCHKA OPOTSCHKA UM EINSACHTDREIN ULL UHRANGETRETEN ANGRIFF INF RGT-

Another convention was to repeat the most important words in the message (again leaving more clues for cryptographers!), and abbreviations were used. So splitting into words and filling out abbreviations:

Aufkl (Aufklärung) abteilung von Kurtinowa nordwestlich Sebez [auf] Fliegerstraße in Richtung Dubrowki, Opotschka. Um 18:30 Uhr angetreten angriff. Inf (Infanterie) Rgt (Regiment)-

English: Reconnaissance division from Kurtinowa north-west of Sebezh on the flight corridor towards Dubrowki, Opochka. Attack begun at 18:30 hours. Infantry Regiment-

Use the code below to make your own Enigma Machine and send coded messages to your friends and family! :D
Rotor ypes I-V are supported and you can have any number of rotors you want! (As long as you specify the rotor positions and ring settings).

In [22]:
# Initialise Enigma Machine
EM = EnigmaMachine(rotor_types=['I', 'II', 'III'],  
                   ring_settings='AAA',
                   rotor_positions='AAA',
                   reflector_mapping='B',
                   steckered_pairing='AB CD EF'
                   )

# Encrypt message
EM.encrypt_message('A really cool message')

'B DUDUIX UMSO NKNFPJJ'